In [7]:
with open("./datasets/ml-latest-small/movies.csv") as f:
    # Each line is of form: <movieId>,<title>,<genres>
    movies = {}
    for line in f:
        split_line = line.strip().split(",")
        
        movies[split_line[0]] = {
            "name": split_line[1],
            "tags": split_line[2].split("|"),
        }
    # first line in file
    del movies["movieId"]

In [9]:
with open("./datasets/ml-latest-small/tags.csv") as f:
    # Each line is of form: <userId>,<movieId>,<tag>,<timestamp>
    # We will only use <movieId>,<tag>
    for line in f:
        split_line = line.strip().split(",")
        if split_line[0] == "userId":
            continue
        movies[split_line[1]]["tags"].append(split_line[2])

In [ ]:
with open("./datasets/ml-latest-small/rating.csv") as f:
    # Each line is of form: <userId>,<movieId>,<rating>,<timestamp>
    # We will only use <userId>,<movieId>,<rating>
    users = {}
    for line in f:
        split_line = line.strip().split(",")
        if split_line[0] == "userId":
            continue
        
        if split_line[0] not in users:
            users[split_line[0]] = {
                "positives": [],
                "negatives": [],
            }
        
        # Assume all ratings above 2.5 (scale 0.5 -> 5) are positive
        if split_line[2] > "2.5":
            users[split_line[0]]["positives"].append(split_line[1])
        else:
            users[split_line[0]]["positives"].append(split_line[1])
        
        